In [1]:
from math import nan
import pandas as pd

unsd_m49 = pd.read_excel('../data/UNSD-M49.xlsx')

In [2]:

def build_region_hierarchy(db, keys):
    key = keys[0]
    keys = keys[1:]
    regions = set(db[key].dropna())
    if len(regions) == 0: return build_region_hierarchy(db, keys)
    if len(keys) == 0: return regions
    d = dict(_type=key)
    for region in regions:
        d[region] = build_region_hierarchy(db[db[key] == region], keys)
    return d

region_hierarchy = build_region_hierarchy(unsd_m49, 'Region Name, Sub-region Name, Intermediate Region Name, Country or Area'.split(', '))

In [62]:
from bokeh.io import output_notebook, show
from bokeh.models import *
from bokeh.layouts import *

output_notebook()

region_levels = []

for level in range(4):
    def set_callback(g, level):
        def callback(attr, old, new):
            next_level = level + 1
            
            db = region_hierarchy
            for l in range(next_level):
                rbg = region_levels[l]
                label = rbg.labels[rbg.active]
                db = db[label]
                if type(db) == set: return
                
                if l == next_level - 1:
                    region_levels[next_level].labels = sorted(db.keys())[:-1]
            
            
            if next_level < len(region_levels): region_levels[next_level].visible = True
            for l in region_levels[next_level+1:]:
                l.visible = False
            
            db = region_hierarchy
            for l in range(next_level):
                rbg = region_levels[l]
                label = rbg.labels[rbg.active]
                db = db[label]
                if l == next_level - 1:
                    region_levels[next_level].labels = sorted(db.keys())[:-1]

        g.on_change('active', callback)
    
    g = RadioButtonGroup(labels=[f'Level {level}', 'A', 'B', 'C'],
                         visible=False if level > 0 else True)
    set_callback(g, level)
    region_levels.append(g)

RegionLevels = Column(*region_levels)

def RegionHierarchySelector(doc):
    region_levels[0].labels = sorted(region_hierarchy.keys())[:-1]
    doc.add_root(RegionLevels)

show(RegionHierarchySelector)

Loading BokehJS ...

In [66]:
rh = region_hierarchy

rh['Asia']['Eastern Asia']

{'China',
 'China, Hong Kong Special Administrative Region',
 'China, Macao Special Administrative Region',
 "Democratic People's Republic of Korea",
 'Japan',
 'Mongolia',
 'Republic of Korea'}

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=1773&bokeh-absolute-url=http://localhost:61258&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:61258', method='GET', uri='/autoload.js?bokeh-autoload-element=1773&bokeh-absolute-url=http://localhost:61258&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/wakita/.venvs/vis19/lib/python3.7/site-packages/tornado/web.py", line 1699, in _execute
    result = await result
  File "/Users/wakita/.venvs/vis19/lib/python3.7/site-packages/tornado/gen.py", line 742, in run
    yielded = self.gen.throw(*exc_info)  # type: ignore
  File "/Users/wakita/.venvs/vis19/lib/python3.7/site-packages/bokeh/server/views/autoload_js_handler.py", line 60, in get
    session = yield self.get_session()
  File "/Users/wakita/.venvs/vis19/lib/python3.7/site-packages/tornado/gen.py", line 735, in run
    value = future.result()
  File "/Users/wakita/.venvs